In [6]:
import pandas as pd
import urllib.request
#-------------------------------------------------------------------------------
from google.colab import drive # somente necessário pra uso no google colab
drive.mount('/content/drive')
path = '/content/drive/My Drive/Colab Notebooks/covid19/brasil io' # DEVE SER MUDADO DE ACORDO COM A APLICAÇÃO (AFETA TODO O RESTO DO CÓDIGO)
# pode ser o caminho raíz do seu projeto
#-------------------------------------------------------------------------------

# PREPARAÇÃO DO DATAFRAME DE REFERÊNCIA 

# carrega os dados do brasil.io
def load_data():
    url = "https://brasil.io/dataset/covid19/caso?format=csv"
    df  = pd.read_csv(url)
    df  = df.fillna(0)
    df  = df.loc[df["city"] != "Importados/Indefinidos"] # retira dados importados/indefinidos
    return df

# carrega os dados de um arquivo raw do github que contém todas as cidades brasileiras
def cidades():
    url_cidades = "https://raw.githubusercontent.com/leo-smi/latitude-e-longitude-estados-e-cidades-brasileiras/master/cidades.csv"
    df_cidades  = pd.read_csv(url_cidades)
    return df_cidades

# teste de aquisição do dataframe
df = load_data()

# Ultimo dia do dataset
max_day = max(df.date)

print(len(df.columns))

df.head()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
11


,date,state,city,place_type,confirmed,deaths,is_last,estimated_population_2019,city_ibge_code,confirmed_per_100k_inhabitants,death_rate
0,2020-04-13,MG,Abaeté,city,1,0.0,True,23237.0,3100203.0,4.30348,0.0
1,2020-04-13,MG,Alfenas,city,1,0.0,True,79996.0,3101607.0,1.25006,0.0
2,2020-04-13,MG,Alpercata,city,1,0.0,True,7424.0,3101805.0,13.46983,0.0
3,2020-04-13,MG,Araguari,city,7,0.0,True,117267.0,3103504.0,5.96928,0.0
4,2020-04-13,MG,Arcos,city,4,0.0,True,40092.0,3104205.0,9.97705,0.0


In [7]:
# CORREÇÃO DIÁRIA DO DATASET TANTO PROS ESTADOS QUANTO PARA OS MUNICÍPIOS (ESSA ETAPA DEMANDA TEMPO)
# ESTA CÉLULA SÓ DEVE SER EXECUTADA A PRIMEIRA VE E QUANDO OS DADOS ESTIVEREM DESATUALIZADOS 

# carrega o dataset do brasil.io (exceto os casos indefinidos/importados)
DF = load_data()
DF = DF.loc[DF['date'] != max_day] # EVITA PEGAR DADOS SENDO ATUALIZADO PELA EQUIPE DO BRASIL.io, OU SEJA, O DIA DE HOJE
# cria uma lista com todos os dias desde o primeiro caso
DATAS = DF.sort_values(by='date', ascending=True).drop_duplicates(subset = 'date')['date'].to_list()
# print(DATAS)
# cria dataframes separados por estados e cidades
DF_estados = DF.loc[DF['place_type'] == 'state']
DF_cidades = DF.loc[DF['place_type'] == 'city']
# cria listas com todos os estados e cidades com casos até o momento
estados = DF_estados.drop_duplicates(subset = ['state']).state.to_list()
cidades = DF_cidades.drop_duplicates(subset = ['city']).city.to_list()

#-------------------------------------------------------------------------------
# FUNÇÕES
#-------------------------------------------------------------------------------

# Função que verifica os dias faltantes dos estados ou cidades
def verifyDays(elementos, key_elemento, DF_elementos): # Ex: verifyDays(estados, 'state', DF_estados)
    for i in range(len(elementos)):
        elemento                  = elementos[i] # estado em estados
        datas_elemento_real       = DF_elementos.loc[DF_elementos[key_elemento] == elemento].date.to_list() # datas que possuem dados do estado
        primeiro_dia_elemento     = min(datas_elemento_real) # primeiro em que foi publicado boletim
        ultimo_dia_elemento       = max(datas_elemento_real) # ultimo dia que os estado publicou boletim
        ultimo_dia_elemento_ideal = max(DATAS) # ultimo dia com casos no dataset
        datas_elemento_ideal      = pd.date_range(start = primeiro_dia_elemento, end = ultimo_dia_elemento_ideal, freq='d').strftime('%Y-%m-%d').to_list()#[0].tolist()
        dias_faltantes_elemento   = sorted(list(set(datas_elemento_ideal) - set(datas_elemento_real))) # resta somente os dias que não foram publicados boletins
        # print(f'Estado/Cidade: {elementos[i]},  Primeiro dia: {primeiro_dia_elemento},  Ultimo dia: {ultimo_dia_elemento},  Dias faltantes: {dias_faltantes_elemento}')

# Função que retorna uma lista de dataframes dos estados ou cidades por dia
def dataframes(DF_elementos): # Ex: dataframes(DFs_estados)
    DFs_elementos = [] 
    for i in range(len(DATAS)):
        DFs_elementos.append(DF_elementos.loc[DF_elementos['date'] == DATAS[i]])
    return DFs_elementos

# Função que preenche as lacunas faltantes dos estados ou cidades
def fillGaps(DFs_elementos, key_elemento, key_value): # Ex fillGaps(DFs_estados, 'state', 'confirmed')
    for i in range(len(DFs_elementos)): 
        if i <= len(DFs_elementos)-2:
            DFs_elementos[i+1] = pd.concat([DFs_elementos[i+1], DFs_elementos[i]])
            DFs_elementos[i+1] = DFs_elementos[i+1].sort_values(by=[key_elemento, key_value], ascending=[True, False])
            DFs_elementos[i+1] = DFs_elementos[i+1].reset_index(drop=True)
            DFs_elementos[i+1] = DFs_elementos[i+1].drop_duplicates(subset=key_elemento, keep='first')
            DFs_elementos[i+1] = DFs_elementos[i+1].sort_values(by=key_value, ascending=False)
            DFs_elementos[i+1] = DFs_elementos[i+1].reset_index(drop=True)
        # corrigir as datas de cada dataframe
        for index, row in DFs_elementos[i].iterrows():
            DFs_elementos[i]['date'][index] = DATAS[i]        

#-------------------------------------------------------------------------------
# CORREÇÃO DOS DADOS ESTADUAIS
#-------------------------------------------------------------------------------

# Verifica os dias faltantes dos estados
verifyDays(estados, 'state', DF_estados)

# lista de dataframes dos estados por dia
DFs_estados = dataframes(DF_estados)

# Aplica a função preencher lacunas nos estados
fillGaps(DFs_estados, 'state', 'confirmed')

# concatenando todos os dataframes dos estados por dia
dfs_estados = pd.concat(DFs_estados)
dfs_estados = dfs_estados.sort_values(by=['date', 'state'], ascending=[True, True])
dfs_estados = dfs_estados.reset_index(drop=True)
# dfs_estados.head(27)
# dfs_estados = dfs_estados.loc[dfs_estados['state'] == "PR"]    
# dfs_estados.plot('date', 'confirmed')

#-------------------------------------------------------------------------------
# CORREÇÃO DOS DADOS MUNICIPAIS
#-------------------------------------------------------------------------------

# Verifica os dias faltantes das cidades
verifyDays(cidades, 'city', DF_cidades)

# lista de dataframes dos estados por dia
DFs_cidades = dataframes(DF_cidades)

# Aplica a função preencher lacunas nos estados
fillGaps(DFs_cidades, 'city', 'confirmed')

# concatenando todos os dataframes dos estados por dia
dfs_cidades = pd.concat(DFs_cidades)
dfs_cidades = dfs_cidades.sort_values(by=['date', 'city'], ascending=[True, True])
dfs_cidades = dfs_cidades.reset_index(drop=True)
# dfs_cidades.head(27)
# dfs_cidades = dfs_cidades.loc[dfs_cidades['city'] == "PR"]    
# dfs_cidades.plot('date', 'confirmed')


# CONCATENANDO OS DATAFRAMES DOS ESTADOS E DAS CIDADES
df_FullData = pd.concat([dfs_estados, dfs_cidades])
df_FullData = df_FullData.reset_index(drop=True)
df_FullData.to_csv(path + '/fullDataFilledGaps.csv', index=False)


df_FullData.tail()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,date,state,city,place_type,confirmed,deaths,is_last,estimated_population_2019,city_ibge_code,confirmed_per_100k_inhabitants,death_rate
12711,2020-04-12,MA,Zé Doca,city,1,0.0,True,51714.0,2114007.0,1.93371,0.0
12712,2020-04-12,GO,Águas Lindas de Goiás,city,2,0.0,True,212440.0,5200258.0,0.94144,0.0
12713,2020-04-12,SC,Águas Mornas,city,1,0.0,True,6469.0,4200606.0,15.45834,0.0
12714,2020-04-12,SP,Águas de Lindóia,city,1,1.0,True,18705.0,3500501.0,5.34616,1.0
12715,2020-04-12,SP,Águas de São Pedro,city,1,0.0,True,3451.0,3500600.0,28.97711,0.0


In [0]:
# SALVANDO POR DIA, o último dia do dataset original brasil.io é evitado

df = pd.read_csv(path + '/fullDataFilledGaps.csv')
datas = df.sort_values(by='date', ascending=True).drop_duplicates(subset = 'date')['date'].to_list()

for data in datas:
    df_data = df.loc[df['date'] == data]
    df_data.to_csv(f"{path}/por datas corrigido sem coordenadas/covid19Br_{data}.csv", index=False)


In [9]:
# -------------------------------------------------------------------------------
# ADICIONANDO LATITUDE E LONGITUDE NO DATASET E SALVAMENTO DO MESMO
# -------------------------------------------------------------------------------

try:
    import geocoder # biblioteca geocoder (neste caso específico utiliza a API do bing maps)
except:
    !pip install geocoder
    import geocoder

# esse token é pessoal, mas pode ser usado para testes, em caso de aplicação deve 
# ser gerado um novo com sua conta no link https://www.bingmapsportal.com/Application
key = 'AiXNSTykHCrpomqs090zP_tNFDTuLzim-yg78FJ4xbUKOnFJ7EAUmjIAVN8QGQ1d' 

# Carrega todo o dataset corrigido
df_fullData = pd.read_csv(path + '/fullDataFilledGaps.csv')
# cria uma lista com as datas 
datas = df_fullData.sort_values(by='date', ascending=True).drop_duplicates(subset = 'date')['date'].to_list()

for data in datas: 
	# carrega cada dataframe individualmente por dia em cada loop
	df_sem = pd.read_csv(f"{path}/por datas corrigido sem coordenadas/covid19Br_{data}.csv") # sem coordenadas
	# verifica se o dataset do respectivo dia já está com as coordenadas preenchidas, caso contrário o dataset é criado
	# para resetar os dados é só apagar os arquivos da pasta com dados
	try:
		df_com = pd.read_csv(f"{path}/por datas corrigido com coordenadas/covid19Br_{data}.csv") # com coordenadas
		pass
	except:
		# Vetor lagitude e longitude que será aplicado como colunas no dataframe
		latlong = []
		# Dicionário para armazenamento temporário de cidades/estados já analisados (evitar retrabalho de requisição do geocoder)
		cidEst = {"cid_est":[], "lat":[], "lng":[]}
		# loop que adiciona as latitudes e longitudes
		for index, row in df_sem.iterrows():
			if row["place_type"] == "state":
				cidest = row["state"]
			elif row["place_type"] == "city":
				cidest = row["city"]
				
			if cidest in cidEst["cid_est"]:
				lat = cidEst["lat"][cidEst["cid_est"].index(cidest)]
				lng = cidEst["lng"][cidEst["cid_est"].index(cidest)]
			else:
				loc = None
				while loc is None:
					try:
						if row["place_type"] == "state":
							loc = geocoder.bing(str(cidest) + ', ' + "BR", key=key)
						elif row["place_type"] == "city":
							loc = geocoder.bing(str(cidest)+', '+str(row['state'])+', '+"BR", key=key)
						lat = loc.json["lat"]
						lng = loc.json["lng"]
						cidEst["lat"] = lat
						cidEst["lng"] = lng
					except:
						print(f"{cidest}: CIDADE/ESTADO NÃO ENCONTRADO")
						print("Tentando novamente!")		
			# a lista latlong recebe uma sub lista  
			latlong.append([lat, lng])
		# o dicionário temporário é transformado em um dataframe de latitudes e longitudes
		df_latLong_full = pd.DataFrame(latlong, columns=['latitude', 'longitude'], index=df_sem.index)
		# o dataframe é incorporado ao dataset
		df_com_new = pd.concat([df_sem, df_latLong_full], axis=1)
		# SALVANDO O DATAFRAME 
		df_com_new.to_csv(f"{path}/por datas corrigido com coordenadas/covid19Br_{data}.csv", index=False)

# concatena todos os dias
dfs = []
for data in datas:
	dfs.append(pd.read_csv(f"{path}/por datas corrigido com coordenadas/covid19Br_{data}.csv"))

df_fullData_new = pd.concat(dfs)
df_fullData_new.to_csv(path + '/dataFullReady.csv', index=False)
df_fullData_new

     |████████████████████████████████| 102kB 3.7MB/s 


,date,state,city,place_type,confirmed,deaths,is_last,estimated_population_2019,city_ibge_code,confirmed_per_100k_inhabitants,death_rate,latitude,longitude
0,2020-02-25,SP,0,state,1,0.0,False,45919049.0,35.0,0.00218,0.0,-23.533970,-46.628868
1,2020-02-25,SP,São Paulo,city,1,0.0,False,12252023.0,3550308.0,0.00816,0.0,-23.562870,-46.654678
0,2020-02-26,SP,0,state,1,0.0,False,45919049.0,35.0,0.00218,0.0,-23.533970,-46.628868
1,2020-02-26,SP,São Paulo,city,1,0.0,False,12252023.0,3550308.0,0.00816,0.0,-23.562870,-46.654678
0,2020-02-27,SP,0,state,1,0.0,False,45919049.0,35.0,0.00218,0.0,-23.533970,-46.628868
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000,2020-04-12,MA,Zé Doca,city,1,0.0,True,51714.0,2114007.0,1.93371,0.0,-3.277780,-45.656288
1001,2020-04-12,GO,Águas Lindas de Goiás,city,2,0.0,True,212440.0,5200258.0,0.94144,0.0,-15.732810,-48.267769
1002,2020-04-12,SC,Águas Mornas,city,1,0.0,True,6469.0,4200606.0,15.45834,0.0,-27.695589,-48.823269
1003,2020-04-12,SP,Águas de Lindóia,city,1,1.0,True,18705.0,3500501.0,5.34616,1.0,-22.474670,-46.630032


In [10]:
# TROCA DAS KEYS DAS COLUNAS

import pandas as pd

# Carrega todo o dataset "corrigido"
def loadFullData(sort_by=None):
    df  = pd.read_csv(path + '/dataFullReady.csv')
    df  = df.fillna(0)
    df  = pd.DataFrame({
        'date':df.date,
        'state':df.state,
        'city':df.city,
        'level':df.place_type,
        'lat':df.latitude,
        'lng':df.longitude,
        'cases':df.confirmed,
        # 'isLast':df.is_last, # não é necessário de foi corrigido as lacunas do dataset
        'pop2019':df.estimated_population_2019,
        'ibgeCode':df.city_ibge_code,
        'casesPer100k':df.confirmed_per_100k_inhabitants,
        'deaths':df.deaths,
        'deathRate':df.death_rate,
    })
    df.to_csv(path + '/dataFullAligned.csv', index=False)
    return df

loadFullData()

,date,state,city,level,lat,lng,cases,pop2019,ibgeCode,casesPer100k,deaths,deathRate
0,2020-02-25,SP,0,state,-23.533970,-46.628868,1,45919049.0,35.0,0.00218,0.0,0.0
1,2020-02-25,SP,São Paulo,city,-23.562870,-46.654678,1,12252023.0,3550308.0,0.00816,0.0,0.0
2,2020-02-26,SP,0,state,-23.533970,-46.628868,1,45919049.0,35.0,0.00218,0.0,0.0
3,2020-02-26,SP,São Paulo,city,-23.562870,-46.654678,1,12252023.0,3550308.0,0.00816,0.0,0.0
4,2020-02-27,SP,0,state,-23.533970,-46.628868,1,45919049.0,35.0,0.00218,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
12693,2020-04-12,MA,Zé Doca,city,-3.277780,-45.656288,1,51714.0,2114007.0,1.93371,0.0,0.0
12694,2020-04-12,GO,Águas Lindas de Goiás,city,-15.732810,-48.267769,2,212440.0,5200258.0,0.94144,0.0,0.0
12695,2020-04-12,SC,Águas Mornas,city,-27.695589,-48.823269,1,6469.0,4200606.0,15.45834,0.0,0.0
12696,2020-04-12,SP,Águas de Lindóia,city,-22.474670,-46.630032,1,18705.0,3500501.0,5.34616,1.0,1.0


In [11]:
# VISUALIZANDO O DATAFRAME DOS ESTADOS

df = loadFullData()
df = df.loc[df['level'] == 'state']
df

,date,state,city,level,lat,lng,cases,pop2019,ibgeCode,casesPer100k,deaths,deathRate
0,2020-02-25,SP,0,state,-23.533970,-46.628868,1,45919049.0,35.0,0.00218,0.0,0.0000
2,2020-02-26,SP,0,state,-23.533970,-46.628868,1,45919049.0,35.0,0.00218,0.0,0.0000
4,2020-02-27,SP,0,state,-23.533970,-46.628868,1,45919049.0,35.0,0.00218,0.0,0.0000
6,2020-02-28,SP,0,state,-23.533970,-46.628868,2,45919049.0,35.0,0.00436,0.0,0.0000
8,2020-02-29,SP,0,state,-23.533970,-46.628868,2,45919049.0,35.0,0.00436,0.0,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...
11715,2020-04-12,RS,0,state,-30.039530,-51.216930,664,11377239.0,43.0,5.83621,16.0,0.0241
11716,2020-04-12,SC,0,state,-27.599840,-48.541821,776,7164788.0,42.0,10.83075,24.0,0.0309
11717,2020-04-12,SE,0,state,-10.910600,-37.059898,44,2298696.0,28.0,1.91413,4.0,0.0909
11718,2020-04-12,SP,0,state,-23.533970,-46.628868,8755,45919049.0,35.0,19.06616,588.0,0.0672


In [12]:
# GRÁFICO DE BARRAS DOS CASOS NOVOS E ÓBITOS NOVOS DO BRASIL

import matplotlib.pyplot as plt

df = loadFullData()
df = df.loc[df['level'] == 'state']

datas = df.sort_values(by='date', ascending=True).drop_duplicates(subset = 'date')['date'].to_list()
estados = df.drop_duplicates(subset='state')['state'].to_list()

# casos acumulados
casos_acumulados = []
for data in datas:
    casos_acumulados.append(df.loc[df['date'] == data].cases.sum())
print(casos_acumulados)
# casos novos
casos_novos = []
anterior = 0
for i in range(len(casos_acumulados)):
    if i == 0:
        casos_novos.append(casos_acumulados[i])
    else:
        casos_novos.append(casos_acumulados[i] - casos_acumulados[i-1])
print(casos_novos)

# óbitos acumulados
obitos_acumulados = []
for data in datas:
    obitos_acumulados.append(df.loc[df['date'] == data].deaths.sum())
print(obitos_acumulados)
# óbitos novos
obitos_novos = []
anterior = 0
for i in range(len(obitos_acumulados)):
    if i == 0:
        obitos_novos.append(obitos_acumulados[i])
    else:
        obitos_novos.append(obitos_acumulados[i] - obitos_acumulados[i-1])
# print(obitos_novos)

# Plotando
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(
    x            = datas,
    y            = casos_novos,
    name         = 'Casos acumulados',
    marker_color = 'indianred',
    line_shape='spline'
))

fig.add_trace(go.Scatter(
    x            = datas,
    y            = obitos_novos,
    name         = 'Mortes acumuladas',
    marker_color = 'blue',
    line_shape='spline'
))
fig.update_traces(mode='lines+markers')
fig.update_layout(xaxis_tickangle=-45, yaxis_type="log", xaxis ={'type': 'category'})#, barmode='group')
fig.show()

[1, 1, 1, 2, 2, 2, 2, 2, 3, 8, 14, 19, 25, 25, 29, 43, 89, 95, 135, 154, 279, 326, 513, 641, 930, 1132, 1594, 1952, 2255, 2561, 2987, 3476, 3953, 4303, 4660, 5824, 6931, 8098, 9212, 10381, 11421, 12261, 14082, 16049, 17919, 19538, 20942, 22456]
[1, 0, 0, 1, 0, 0, 0, 0, 1, 5, 6, 5, 6, 0, 4, 14, 46, 6, 40, 19, 125, 47, 187, 128, 289, 202, 462, 358, 303, 306, 426, 489, 477, 350, 357, 1164, 1107, 1167, 1114, 1169, 1040, 840, 1821, 1967, 1870, 1619, 1404, 1514]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 3.0, 7.0, 11.0, 18.0, 25.0, 34.0, 47.0, 59.0, 77.0, 93.0, 115.0, 137.0, 166.0, 202.0, 244.0, 305.0, 365.0, 445.0, 496.0, 569.0, 691.0, 809.0, 942.0, 1045.0, 1137.0, 1237.0]


In [13]:
# GRÁFICO DE BARRAS DOS CASOS E ÓBITOS ACUMULADOS DO BRASIL

import matplotlib.pyplot as plt

df = loadFullData()
df = df.loc[df['level'] == 'state']

datas = df.sort_values(by='date', ascending=True).drop_duplicates(subset = 'date')['date'].to_list()
estados = df.drop_duplicates(subset='state')['state'].to_list()

casos_acumulados = []
for data in datas:
    casos_acumulados.append(df.loc[df['date'] == data].cases.sum())

obitos_acumulados = []
for data in datas:
    obitos_acumulados.append(df.loc[df['date'] == data].deaths.sum())

# Plotando

import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Bar(
    x            = datas,
    y            = casos_acumulados,
    name         = 'Casos acumulados',
    marker_color = 'indianred'
))

fig.add_trace(go.Bar(
    x            = datas,
    y            = obitos_acumulados,
    name         = 'Mortes acumuladas',
    marker_color = 'yellow'
))

fig.update_layout(xaxis_tickangle=-90, yaxis_type="log", barmode='group', xaxis ={'type': 'category'})
fig.show()

In [14]:
# GRÁFICO DE BARRAS DOS CASOS E ÓBITOS ACUMULADOS DE UM DETERMINADO ESTADO

# 
df = loadFullData()
# df = df.loc[df['date'] == max(df['date'])]
df = df.loc[df['level'] == 'state']
df = df.loc[df['state'] == 'SP']
# df = df.loc[df['isLast'] == True]
df = df.sort_values(by='date', ascending=True)
df = df.reset_index(drop=True)

import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Bar(
    x            = df["date"],
    y            = df["cases"],
    name         = 'Casos acumulados',
    marker_color = 'indianred'
))

fig.add_trace(go.Bar(
    x            = df["date"],
    y            = df["deaths"],
    name         = 'Mortes acumuladas',
    marker_color = 'yellow'
))


fig.update_layout(xaxis_tickangle=-90, yaxis_type="log", barmode='group',xaxis ={'type': 'category'})
fig.show()

In [15]:
# GRÁFICO DE BARRAS DOS CASOS E ÓBITOS ACUMULADOS DE UMA DETERMINADA CIDADE

# Ultimo dia
df = loadFullData()
# df = df.loc[df['date'] == max(df['date'])]
df = df.loc[df['level'] == 'city']
df = df.loc[df['city'] == 'São Paulo']
# df = df.loc[df['isLast'] == True]
df = df.sort_values(by='date', ascending=True)
df = df.reset_index(drop=True)

import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Bar(
    x            = df["date"],
    y            = df["cases"],
    name         = 'Casos acumulados',
    marker_color = 'indianred'
))

fig.add_trace(go.Bar(
    x            = df["date"],
    y            = df["deaths"],
    name         = 'Mortes acumuladas',
    marker_color = 'yellow'
))


fig.update_layout(xaxis_tickangle=-45, yaxis_type="log", barmode='group',xaxis ={'type': 'category'})
fig.show()

In [16]:
# SOMATÓRIO DOS CASOS
df = loadFullData()

max_date = max(df.date.to_list())
print(max_date)
df = df.loc[df['date'] == max_date]
df = df.loc[df['level'] == 'state']
soma = df.cases.sum()
soma

2020-04-12


22456

In [18]:
# CORRELAÇÃO CRUZADA (teste)
import numpy as np

for delay in range(0, 10):
    corr = np.correlate([1, 2, 3], [1+delay, 2+delay, 3+delay])
    print(f"Delay: {delay}, Correlação cruzada: {corr[0]}")

Delay: 0, Correlação cruzada: 14
Delay: 1, Correlação cruzada: 20
Delay: 2, Correlação cruzada: 26
Delay: 3, Correlação cruzada: 32
Delay: 4, Correlação cruzada: 38
Delay: 5, Correlação cruzada: 44
Delay: 6, Correlação cruzada: 50
Delay: 7, Correlação cruzada: 56
Delay: 8, Correlação cruzada: 62
Delay: 9, Correlação cruzada: 68
